# Volunteer model loss calculation

This notebook takes a `subject_id` and calculates the MSE of each volunteer's model to the galaxy data. These losses are then glued to the notebook for use elsewhere.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import json
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
from gzbuilder_analysis import parsing, rendering, fitting
import lib.galaxy_utilities as gu
import scrapbook as sb

/opt/apps/pkgs/anaconda3/2019.03/intel64/lib/python3.7/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}


In [3]:
subject_id = 20902040

In [4]:
# Parameters
subject_id = 21686551


In [5]:
diff_data_df = pd.read_pickle('lib/fitting_metadata.pkl')

In [6]:
def get_losses_by_user(subject_id):
  diff_data = diff_data_df.loc[subject_id]
  psf = diff_data['psf']
  pixel_mask = np.array(diff_data['pixel_mask'])[::-1]
  galaxy_data = np.array(diff_data['galaxy_data'])[::-1]
  image_size = galaxy_data.shape[0]
  size_diff = diff_data['size_diff']
  
  cls = gu.classifications.query('subject_ids == {}'.format(subject_id))
  names = cls.user_name 
  annotations = cls['annotations'].apply(json.loads)

  models = annotations.apply(
    parsing.parse_annotation,
    size_diff=size_diff
  )

  tqdm.pandas(
    desc='Rendering models'.format(subject_id),
    leave=False
  )
  rendered = models.progress_apply(
    rendering.calculate_model,
    image_size=image_size,
    psf=psf
  ).rename('rendered')

  tqdm.pandas(
    desc='Calculating differences'.format(subject_id),
    leave=False
  )
  losses = rendered.apply(
    fitting.loss,
    args=(galaxy_data, pixel_mask)
  ).rename('loss')
  all_losses = pd.Series(
    losses.values,
    index=names.values
  ).rename(subject_id)
  best_losses = pd.concat((
    all_losses.where(
      ~all_losses.duplicated(keep=False)
    ).dropna(), # all non-duplicates
    all_losses.where(
      all_losses.duplicated(keep=False)
    ).dropna().groupby(level=0).min() # best value of duplicates
  ), axis=0)
  return best_losses

In [7]:
losses = get_losses_by_user(subject_id)
losses

Rendering models:   0%|          | 0/30 [00:00<?, ?it/s]

Rendering models:   7%|▋         | 2/30 [00:04<01:05,  2.32s/it]

Rendering models:  10%|█         | 3/30 [00:04<00:45,  1.69s/it]

Rendering models:  13%|█▎        | 4/30 [00:06<00:42,  1.65s/it]

Rendering models:  17%|█▋        | 5/30 [00:06<00:31,  1.25s/it]

...

Rendering models:  87%|████████▋ | 26/30 [00:12<00:01,  3.89it/s]

Rendering models:  90%|█████████ | 27/30 [00:12<00:00,  3.91it/s]

Rendering models:  97%|█████████▋| 29/30 [00:12<00:00,  4.70it/s]

Rendering models: 100%|██████████| 30/30 [00:12<00:00,  4.95it/s]

ethanscamander                        0.001751
not-logged-in-fceaf6f9645a2f020c34    0.001514
ElisabethB                            0.015646
not-logged-in-f7e2260ed4a9517a3e4a    0.040440
cschwefl                              0.000180
CThomas                               0.010729
not-logged-in-0b3c4d412fae103c1ef6    0.036505
yooman12                              0.012277
kjkavene                              0.000115
not-logged-in-c3e2bb63f9ec3bdeea91    0.000437
Stella07                              0.000155
raisamaduro                           0.002271
dxjdxjhhhh                            0.000985
crush202020                           0.000097
pangeli5                              0.000220
acapirala                             0.000377
Lavadude                              0.002794
Tessaje04                             0.005374
not-logged-in-37e203002afa1179a0e6    0.788187
mikedyke                              0.000428
McMaster.Sam                          0.000284
cooper.hull  

In [8]:
sb.glue('subject_id', subject_id)
sb.glue('losses', losses.to_dict())